In [ ]:
import wrds       
conn = wrds.Connection() 

df = conn.raw_sql(" select gvkey, datadate,  " 
                        + variables +
                      " from comp.funda "
                      " where datadate >= '" + startdate + "' "
                      " and indfmt='INDL' and datafmt='STD' and popsrc='D' and consol='C' ", date_cols=['datadate'])
    df.gvkey = df.gvkey.astype(int)
    df = df.sort_values(by=['gvkey','datadate'])
    return df.set_index(['gvkey','datadate'])

comp = conn.raw_sql(
    """
    select gvkey, datadate, at
    from comp.funda
    where datadate >= '2000-01-01' and at>0 
    and indfmt='INDL' and datafmt='STD' and popsrc='D' and consol='C'
    order by gvkey, datadate
    """, 
    date_cols=['datadate']
)

# convert string or float to int
comp.gvkey = comp.gvkey.astype(int)
    
def coalesce(df, cols) :
        return df[cols].bfill(axis=1).iloc[:, 0]
        
def bookequity(df) :
    deferredTaxes = df.txditc.fillna(0)    
    preferredStock = coalesce(df,['pstkrv','pstkl','pstk']).fillna(0)                  
    df['seq2'] = df.ceq + df.pstk.fillna(0)                
    df['seq3'] = np.where((df['at']>=0) & (df['lt']>=0), df['at']-df['lt'], np.nan)        
    shareholdersEquity = coalesce(df,['seq','seq2','seq3'])
    bookEquity = shareholdersEquity + deferredTaxes - preferredStock
    bookEquity = np.where(bookEquity>0,bookEquity,np.nan)
    return pd.Series(bookEquity, index=df.index)